# EasyOCR 訓練與改善指南

## 🎯 目標
改善 EasyOCR 對台灣發票/收據的繁體中文辨識準確度

## ⚠️ 重要提醒
**訓練 EasyOCR 是一個非常複雜且耗時的過程,需要:**
- 大量標註數據 (數千到數萬張圖片)
- 強大的 GPU 資源 (至少 RTX 3090 或更好)
- 深度學習專業知識
- 數天到數週的訓練時間

## 📋 本指南包含:

### 方案 A: 不訓練的改善方法 (推薦)
1. 參數調優
2. 圖片預處理
3. 後處理與修正
4. 使用多個 OCR 引擎

### 方案 B: 使用遷移學習 (中等難度)
5. Fine-tune 現有模型

### 方案 C: 完整訓練 (最複雜)
6. 從頭訓練 EasyOCR

---

## 方案 A1: 參數調優 (最簡單,最有效)

**不需要訓練,只需調整 EasyOCR 的參數**

這是最快速且有效的改善方法!

In [2]:
# 方案 A1: 調整 EasyOCR 參數以改善辨識

import easyocr
import cv2
import numpy as np

# 初始化 reader
reader = easyocr.Reader(['ch_tra', 'en'], gpu=True)

# 測試不同參數組合
image_path = './samples/2.jpeg'

print("="*60)
print("測試 1: 預設參數")
print("="*60)
result1 = reader.readtext(image_path, detail=1)
for bbox, text, conf in result1:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("測試 2: 調整文字檢測閾值")
print("="*60)
result2 = reader.readtext(
    image_path,
    detail=1,
    text_threshold=0.6,      # 降低 (預設 0.7) - 更容易偵測文字
    low_text=0.3,            # 降低 (預設 0.4) - 檢測更多文字區域
    link_threshold=0.3,      # 降低 (預設 0.4) - 更容易連接文字
)
for bbox, text, conf in result2:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("測試 3: 調整圖片大小參數")
print("="*60)
result3 = reader.readtext(
    image_path,
    detail=1,
    canvas_size=2560,        # 增加 (預設 2560) - 處理更大的圖片
    mag_ratio=1.5,           # 增加 (預設 1) - 放大文字
)
for bbox, text, conf in result3:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("測試 4: 綜合優化參數")
print("="*60)
result4 = reader.readtext(
    image_path,
    detail=1,
    paragraph=False,         # 不合併段落
    min_size=10,             # 最小文字大小
    text_threshold=0.6,      # 文字檢測閾值
    low_text=0.3,            # 低文字閾值
    link_threshold=0.3,      # 連接閾值
    canvas_size=2560,        # 畫布大小
    mag_ratio=1.5,           # 放大比例
    contrast_ths=0.1,        # 對比度閾值
    adjust_contrast=0.5,     # 對比度調整
    width_ths=0.7,           # 寬度閾值
)
for bbox, text, conf in result4:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("參數說明:")
print("="*60)
print("""
1. text_threshold (0-1): 文字檢測信心度閾值
   - 降低 → 檢測更多文字 (可能增加誤判)
   - 提高 → 只檢測高信心度文字

2. low_text (0-1): 低信心度文字區域閾值
   - 降低 → 檢測更多模糊文字

3. link_threshold (0-1): 文字連接閾值
   - 降低 → 更容易連接分開的字符

4. canvas_size: 內部處理的圖片大小
   - 增加 → 保留更多細節 (但速度變慢)

5. mag_ratio: 圖片放大比例
   - 增加 → 小字更清晰

6. min_size: 最小文字大小 (像素)
   - 降低 → 檢測更小的文字
""")

測試 1: 預設參數


/Users/wing199901/GitHub/SHape/smart-spend-backend/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[0.99] 韓餐韓國料理
[0.64] 九龍青衣細山路2號美景花園平壼72號舖
[0.81] Te1;2495-0822
[0.42] T1 Io
[1.00] 5
[1.00] Cardno
[1.00] 800012
[0.96] Staff
[1.00] 經理
[0.65] Guest Ilo.
[0.61] Date: 2025-09-26
[0.49] Time;
[0.50] 18;38
[0.68] 17;50 C01牛骨湯飯
[0.92] 98.0
[0.93] 17;50 C04辣泡菜湯飯
[0.71] 18.0
[0.99] 17;50
[0.84] CO5軟滑豆腐海鮮湯
[0.81] 滿]辣)
[0.83] 78.0
[0.86] 17;50 D08柴菜卷飯
[0.82] 62.0
[0.90] Sub Tota1
[0.59] 315.0
[0.98] (0000119183)
[0.62] 10% Service:
[0.66] 31,60
[0.54] Tota1
[0.80] $348.0
[0.81] Tota1
[0.99] $348.0
[1.00] Cash
[1.00] $348.0
[0.97] Helcome

測試 2: 調整文字檢測閾值


/Users/wing199901/GitHub/SHape/smart-spend-backend/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[1.00] 韓餐韓國料理
[0.54] 九龍青衣細山路2號美景花園平耋72號舖
[0.76] Te1;2495-0822
[0.43] T1 Io.
[1.00] 5
[0.68] Cardno; 800012
[0.96] Staff: 經理
[0.89] Guest Ilo.
[0.99] 2
[0.63] Date: 2025-09-25
[0.57] Time: 18:38
[0.88] 17;50 C01牛骨湯飯
[1.00] 98.0
[0.96] 17;50 C04辣泡菜湯飯
[0.82] 78.0
[0.94] 17;50 C05軟滑豆腐海鮮湯
[0.85] 滿]辣)
[0.50] 18.0
[0.48] 17;50 D08柴菜卷飯
[0.75] 62.0
[0.88] Sub Tota1
[1.00] 316.0
[0.96] (0000119183)
[0.55] 10% Service:
[0.58] 31,60
[0.95] Total
[0.99] $348.0
[0.69] Tota1
[0.99] $348.0
[1.00] Cash
[0.89] $348.0
[0.97] Helcome

測試 3: 調整圖片大小參數


/Users/wing199901/GitHub/SHape/smart-spend-backend/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[0.81] 韓餐韓國料理
[0.79] 九龍青衣細山路2號美景花園平耋72號舖
[0.65] Te1;2495-0822
[0.90] #;
[1.00] 5
[0.63] Cardno; 80oo12
[0.95] Staff: 經理
[0.63] Guest Ilo.;
[0.31] Date: 2025-09-25
[0.95] Time;
[0.50] 18;38
[0.60] 17;50 C01牛骨湯飯
[0.99] 98.0
[0.92] 17;50 C04辣泡菜湯飯
[0.90] 18.0
[0.93] 17;50 C05軟滑豆腐海鮮湯
[0.87] 滿]辣)
[0.48] 78.0
[0.51] 17;50 008柴菜卷飯
[0.56] 62,0
[0.91] Sub Tota1
[0.39] 316.0
[0.99] (0000119183)
[0.64] 10% Service:
[0.42] 31 ,60
[0.63] Tota1
[0.96] $348.0
[0.56] Total
[0.64] $348.0
[1.00] Cash
[0.97] $348.0
[0.56] HeIcome

測試 4: 綜合優化參數


/Users/wing199901/GitHub/SHape/smart-spend-backend/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[0.98] 韓餐韓國料理
[0.66] 九龍青衣細山路2號美景花園平壼72號舖
[0.87] Te1;2495-0822
[1.00] 5
[0.74] Cardno; 800o12
[0.88] Staff: 經理
[0.46] Guest llo.: 2
[0.46] Date: 2025-09-26
[0.54] Time; 18:38
[0.83] 17;50 C01牛骨湯飯
[0.99] 98.0
[0.98] 17;50 C04辣泡菜湯飯
[0.61] 78.0
[0.50] 17;50 C05軟滑豆腐海鮮湯
[0.87] 滿]辣)
[0.62] 18.0
[0.47] 17;50 D08紫菜卷飯
[0.83] 62.0
[0.64] Sub Tota1
[0.95] 316.0
[0.95] (0000119183)
[0.62] 10% Service:
[0.64] 31 ,60
[0.92] Total
[0.98] $348.0
[0.63] Total
[0.98] $348.0
[1.00] Cash
[0.85] $348.0
[0.99] Helcome

參數說明:

1. text_threshold (0-1): 文字檢測信心度閾值
   - 降低 → 檢測更多文字 (可能增加誤判)
   - 提高 → 只檢測高信心度文字

2. low_text (0-1): 低信心度文字區域閾值
   - 降低 → 檢測更多模糊文字

3. link_threshold (0-1): 文字連接閾值
   - 降低 → 更容易連接分開的字符

4. canvas_size: 內部處理的圖片大小
   - 增加 → 保留更多細節 (但速度變慢)

5. mag_ratio: 圖片放大比例
   - 增加 → 小字更清晰

6. min_size: 最小文字大小 (像素)
   - 降低 → 檢測更小的文字



---
## 方案 A2: 圖片預處理優化 (重要!)

針對台灣發票的特性進行預處理,可以顯著改善辨識率。

In [4]:
# 方案 A2: 針對發票的圖片預處理

import cv2
import numpy as np
import easyocr

def preprocess_receipt(image_path):
    """針對台灣發票優化的預處理流程"""
    
    # 讀取圖片
    img = cv2.imread(image_path)
    
    # 1. 轉灰階
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 2. 去噪 - 使用雙邊濾波器保留邊緣
    denoised = cv2.bilateralFilter(gray, 9, 75, 75)
    
    # 3. 對比度增強 - CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(denoised)
    
    # 4. 銳化
    kernel_sharpening = np.array([[-1,-1,-1],
                                   [-1, 9,-1],
                                   [-1,-1,-1]])
    sharpened = cv2.filter2D(enhanced, -1, kernel_sharpening)
    
    # 5. 自適應二值化
    binary = cv2.adaptiveThreshold(
        sharpened,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        11,  # block size
        2    # C constant
    )
    
    return img, gray, denoised, enhanced, sharpened, binary

# 測試預處理效果
image_path = './samples/2.jpeg'
reader = easyocr.Reader(['ch_tra', 'en'], gpu=True)

print("="*60)
print("原始圖片 OCR")
print("="*60)
result_original = reader.readtext(image_path, detail=1)
for bbox, text, conf in result_original:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

# 預處理後測試
img, gray, denoised, enhanced, sharpened, binary = preprocess_receipt(image_path)

print("\n" + "="*60)
print("預處理後 OCR (使用 CLAHE 增強)")
print("="*60)
result_enhanced = reader.readtext(enhanced, detail=1)
for bbox, text, conf in result_enhanced:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("預處理後 OCR (使用銳化)")
print("="*60)
result_sharpened = reader.readtext(sharpened, detail=1)
for bbox, text, conf in result_sharpened:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("預處理後 OCR (使用二值化)")
print("="*60)
result_binary = reader.readtext(binary, detail=1)
for bbox, text, conf in result_binary:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

# 保存預處理結果供視覺檢查
cv2.imwrite('./output/preprocessed_enhanced.jpeg', enhanced)
cv2.imwrite('./output/preprocessed_sharpened.jpeg', sharpened)
cv2.imwrite('./output/preprocessed_binary.jpeg', binary)

print("\n預處理圖片已保存至 ./output/ 資料夾")

原始圖片 OCR


/Users/wing199901/GitHub/SHape/smart-spend-backend/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[0.99] 韓餐韓國料理
[0.64] 九龍青衣細山路2號美景花園平壼72號舖
[0.81] Te1;2495-0822
[0.42] T1 Io
[1.00] 5
[1.00] Cardno
[1.00] 800012
[0.96] Staff
[1.00] 經理
[0.65] Guest Ilo.
[0.61] Date: 2025-09-26
[0.49] Time;
[0.50] 18;38
[0.68] 17;50 C01牛骨湯飯
[0.92] 98.0
[0.93] 17;50 C04辣泡菜湯飯
[0.71] 18.0
[0.99] 17;50
[0.84] CO5軟滑豆腐海鮮湯
[0.81] 滿]辣)
[0.83] 78.0
[0.86] 17;50 D08柴菜卷飯
[0.82] 62.0
[0.90] Sub Tota1
[0.59] 315.0
[0.98] (0000119183)
[0.62] 10% Service:
[0.66] 31,60
[0.54] Tota1
[0.80] $348.0
[0.81] Tota1
[0.99] $348.0
[1.00] Cash
[1.00] $348.0
[0.97] Helcome

預處理後 OCR (使用 CLAHE 增強)
[0.70] 韓餐韓國料理
[0.44] 九龍青衣細山路2號美景花園平耋臺72號舖
[0.51] Te1 :2495-0822
[0.49] T1 No
[1.00] 5
[0.84] Cardno
[1.00] 800012
[1.00] Staff
[1.00] 經理
[0.63] Guest Mo.
[0.65] Date: 2025-09-26
[0.70] Time;
[0.68] 18;38
[0.53] 17;50 C01牛骨湯飯
[0.99] 98.0
[0.92] 17;50 C04辣泡菜湯飯
[0.63] 78.0
[0.96] 17;50 C05軟滑豆腐海鮮湯
[1.00] 滿]辣)
[0.76] 18.0
[0.90] 17;50 D08柴菜卷飯
[0.96] 62.0
[0.85] Sub Tota1
[0.94] 316.0
[1.00] (0000119183)
[0.76] 10 Service:
[0.78] 31.60
[0.70]

---
## 方案 A3: 結果後處理與錯誤修正

針對常見的辨識錯誤進行自動修正。

In [ ]:
# 方案 A3: OCR 結果後處理與錯誤修正

import re
from difflib import get_close_matches

# 1. 常見的 OCR 辨識錯誤對照表 (針對台灣發票)
COMMON_ERRORS = {
    # 數字容易辨識錯誤
    'O': '0',  # 英文 O → 數字 0
    'o': '0',
    'I': '1',  # 英文 I → 數字 1
    'l': '1',  # 小寫 L → 數字 1
    'S': '5',
    's': '5',
    'Z': '2',
    'B': '8',
    
    # 中文字容易辨識錯誤
    '辨': '辦',
    '識': '认',
}

# 2. 台灣發票常見關鍵字字典 (用於信心度低的結果)
RECEIPT_KEYWORDS = [
    '統一編號', '發票號碼', '隨機碼', '總計', '營業人', 
    '買受人', '日期', '時間', '品名', '數量', '單價', '金額',
    '小計', '合計', '應稅', '免稅', '課稅別', '稅額',
    '電子發票', '載具', 'QR Code'
]

# 3. 數字格式修正 (發票號碼、金額等)
def fix_invoice_number(text):
    """修正發票號碼格式 (兩個英文字母 + 8位數字)"""
    # 移除空格和特殊符號
    cleaned = re.sub(r'[^A-Z0-9]', '', text.upper())
    
    # 匹配格式: XX-12345678 或 XX12345678
    match = re.match(r'([A-Z]{2})(\d{8})', cleaned)
    if match:
        return f"{match.group(1)}-{match.group(2)}"
    return text

def fix_amount(text):
    """修正金額格式"""
    # 移除非數字和小數點
    cleaned = re.sub(r'[^\d.]', '', text)
    
    # 確保只有一個小數點
    parts = cleaned.split('.')
    if len(parts) > 2:
        cleaned = parts[0] + '.' + ''.join(parts[1:])
    
    try:
        return f"${float(cleaned):,.0f}"
    except:
        return text

def fix_date(text):
    """修正日期格式"""
    # 匹配常見日期格式
    match = re.search(r'(\d{3,4})[年/-]?(\d{1,2})[月/-]?(\d{1,2})', text)
    if match:
        year, month, day = match.groups()
        # 民國年轉西元年
        if len(year) == 3:
            year = str(int(year) + 1911)
        return f"{year}/{month.zfill(2)}/{day.zfill(2)}"
    return text

def correct_ocr_result(text, confidence):
    """
    綜合修正 OCR 結果
    
    Args:
        text: OCR 辨識出的文字
        confidence: 信心度
    
    Returns:
        修正後的文字
    """
    
    # 1. 如果信心度很高 (>0.9),直接返回
    if confidence > 0.9:
        return text
    
    # 2. 常見錯誤替換
    corrected = text
    for wrong, correct in COMMON_ERRORS.items():
        corrected = corrected.replace(wrong, correct)
    
    # 3. 嘗試匹配關鍵字 (信心度 < 0.7)
    if confidence < 0.7:
        matches = get_close_matches(corrected, RECEIPT_KEYWORDS, n=1, cutoff=0.6)
        if matches:
            corrected = matches[0]
    
    # 4. 格式特定修正
    if re.search(r'[A-Z]{2}.*\d{8}', corrected):
        corrected = fix_invoice_number(corrected)
    elif re.search(r'\d+[元$]?', corrected):
        corrected = fix_amount(corrected)
    elif '年' in corrected or '月' in corrected:
        corrected = fix_date(corrected)
    
    return corrected

# 測試後處理效果
import easyocr

reader = easyocr.Reader(['ch_tra', 'en'], gpu=False)
image_path = './samples/2.jpeg'

print("="*60)
print("原始 OCR 結果")
print("="*60)
results = reader.readtext(image_path, detail=1)
for bbox, text, conf in results:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("後處理修正後")
print("="*60)
for bbox, text, conf in results:
    if conf > 0.3:
        corrected = correct_ocr_result(text, conf)
        if text != corrected:
            print(f"[{conf:.2f}] {text} → {corrected} ✓")
        else:
            print(f"[{conf:.2f}] {corrected}")

# 5. 信心度過濾
print("\n" + "="*60)
print("高信心度結果 (>0.7)")
print("="*60)
high_conf_results = [(bbox, correct_ocr_result(text, conf), conf) 
                     for bbox, text, conf in results if conf > 0.7]
for bbox, text, conf in high_conf_results:
    print(f"[{conf:.2f}] {text}")

---
## 方案 A4: 多模型整合 (Ensemble)

結合多個 OCR 引擎的結果,提高準確率。

In [ ]:
# 方案 A4: 多 OCR 引擎整合

import easyocr
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from collections import defaultdict
import numpy as np

def ocr_ensemble(image_path, confidence_threshold=0.5):
    """
    整合多個 OCR 引擎的結果
    
    策略:
    1. 使用 EasyOCR 和 DocTR 兩個引擎
    2. 比較兩個引擎的結果
    3. 若結果一致 → 高信心度
    4. 若結果不一致 → 選擇信心度較高的
    """
    
    results = {}
    
    # 1. EasyOCR 結果
    print("正在執行 EasyOCR...")
    reader = easyocr.Reader(['ch_tra', 'en'], gpu=False)
    easyocr_results = reader.readtext(image_path, detail=1)
    
    results['easyocr'] = []
    for bbox, text, conf in easyocr_results:
        if conf > confidence_threshold:
            results['easyocr'].append({
                'text': text,
                'confidence': conf,
                'bbox': bbox
            })
    
    # 2. DocTR 結果
    print("正在執行 DocTR...")
    model = ocr_predictor(pretrained=True)
    doc = DocumentFile.from_images(image_path)
    doctr_result = model(doc)
    
    results['doctr'] = []
    for page in doctr_result.pages:
        for block in page.blocks:
            for line in block.lines:
                for word in line.words:
                    if word.confidence > confidence_threshold:
                        results['doctr'].append({
                            'text': word.value,
                            'confidence': word.confidence,
                            'bbox': word.geometry
                        })
    
    # 3. 整合結果
    print("\n" + "="*60)
    print("整合結果分析")
    print("="*60)
    
    # 統計每個文字出現的次數和平均信心度
    text_stats = defaultdict(lambda: {'count': 0, 'total_conf': 0, 'sources': []})
    
    for source, items in results.items():
        for item in items:
            text = item['text'].strip()
            text_stats[text]['count'] += 1
            text_stats[text]['total_conf'] += item['confidence']
            text_stats[text]['sources'].append(source)
    
    # 計算最終結果
    final_results = []
    for text, stats in text_stats.items():
        avg_conf = stats['total_conf'] / stats['count']
        
        # 如果兩個引擎都辨識出同樣的文字 → 高信心度
        boost_factor = 1.2 if stats['count'] > 1 else 1.0
        final_conf = min(avg_conf * boost_factor, 1.0)
        
        final_results.append({
            'text': text,
            'confidence': final_conf,
            'agreement': stats['count'] > 1,
            'sources': stats['sources']
        })
    
    # 依信心度排序
    final_results.sort(key=lambda x: x['confidence'], reverse=True)
    
    return final_results, results

# 執行整合
image_path = './samples/2.jpeg'
final_results, raw_results = ocr_ensemble(image_path, confidence_threshold=0.5)

# 顯示各引擎原始結果
print("\n" + "="*60)
print("EasyOCR 原始結果")
print("="*60)
for item in raw_results['easyocr']:
    print(f"[{item['confidence']:.2f}] {item['text']}")

print("\n" + "="*60)
print("DocTR 原始結果")
print("="*60)
for item in raw_results['doctr']:
    print(f"[{item['confidence']:.2f}] {item['text']}")

# 顯示整合後結果
print("\n" + "="*60)
print("整合後的最終結果")
print("="*60)
for item in final_results:
    agreement_mark = "✓✓" if item['agreement'] else "✓"
    sources = "+".join(item['sources'])
    print(f"[{item['confidence']:.2f}] {agreement_mark} {item['text']} ({sources})")

# 只顯示高信心度 (>0.7) 且兩個引擎都認同的結果
print("\n" + "="*60)
print("高信心度且雙引擎認同的結果 (最可靠)")
print("="*60)
reliable_results = [r for r in final_results if r['confidence'] > 0.7 and r['agreement']]
for item in reliable_results:
    print(f"[{item['confidence']:.2f}] {item['text']}")

---
## 方案 B: Fine-tuning (微調現有模型)

⚠️ **難度: 中等** | 需要一些標註資料 (50-200 張) | 訓練時間: 數小時

如果上述方法都無法達到理想效果,可以考慮微調 EasyOCR 的現有模型。

### 優點
- 比從頭訓練快得多
- 需要的資料量較少
- 可以針對特定領域 (台灣發票) 優化

### 步驟概述

In [ ]:
# 方案 B: Fine-tuning 準備工作

"""
EasyOCR Fine-tuning 流程:

1. 準備資料集
   - 收集 50-200 張發票圖片
   - 標註正確的文字位置和內容
   - 格式: 圖片 + 對應的文字標註 (JSON 或 TXT)

2. 使用 EasyOCR 的訓練腳本
   - GitHub: https://github.com/JaidedAI/EasyOCR
   - 需要 PyTorch 和 GPU

3. 訓練參數調整
   - Learning rate: 0.0001 (較小,避免破壞預訓練權重)
   - Batch size: 8-16
   - Epochs: 10-50
   - Early stopping

4. 評估和測試
"""

# 資料標註工具推薦
print("="*60)
print("資料標註工具推薦")
print("="*60)
print("""
1. **Label Studio** (推薦)
   - 網址: https://labelstud.io/
   - 免費開源
   - 支援 OCR 標註
   - 可匯出多種格式
   
   安裝:
   pip install label-studio
   label-studio start

2. **CVAT** (Computer Vision Annotation Tool)
   - 網址: https://www.cvat.ai/
   - Intel 開發
   - 支援文字辨識標註
   
3. **LabelImg**
   - 簡單易用
   - 適合小規模標註
   
   安裝:
   pip install labelImg
   labelImg

4. **MakeSense.ai**
   - 網址: https://www.makesense.ai/
   - 線上工具,無需安裝
   - 適合快速標註
""")

# 資料集格式範例
print("\n" + "="*60)
print("資料集格式範例")
print("="*60)
print("""
資料夾結構:
dataset/
├── images/
│   ├── receipt_001.jpg
│   ├── receipt_002.jpg
│   └── ...
└── labels/
    ├── receipt_001.txt
    ├── receipt_002.txt
    └── ...

標註檔案格式 (receipt_001.txt):
x1,y1,x2,y2,x3,y3,x4,y4,text
100,50,300,50,300,80,100,80,統一編號
100,90,300,90,300,120,100,120,12345678
...

或使用 JSON 格式:
{
  "image": "receipt_001.jpg",
  "annotations": [
    {
      "bbox": [[100,50], [300,50], [300,80], [100,80]],
      "text": "統一編號",
      "confidence": 1.0
    },
    ...
  ]
}
""")

# 快速標註技巧
print("\n" + "="*60)
print("快速標註技巧")
print("="*60)
print("""
1. **先用 EasyOCR 預標註**
   - 用現有模型辨識
   - 只修正錯誤的部分
   - 節省大量時間

2. **只標註常見錯誤區域**
   - 不需要標註整張發票
   - 專注於容易辨識錯誤的字段
   - 例如: 金額、日期、統編

3. **使用模板**
   - 台灣發票格式相對固定
   - 建立模板快速標註
""")

# 預標註腳本範例
print("\n" + "="*60)
print("預標註腳本範例 (節省標註時間)")
print("="*60)

import easyocr
import json
import os

def pre_annotate_images(image_folder, output_folder):
    """
    使用 EasyOCR 預標註圖片
    人工只需要檢查和修正錯誤
    """
    
    reader = easyocr.Reader(['ch_tra', 'en'], gpu=False)
    
    os.makedirs(output_folder, exist_ok=True)
    
    for image_file in os.listdir(image_folder):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        
        image_path = os.path.join(image_folder, image_file)
        results = reader.readtext(image_path, detail=1)
        
        annotations = []
        for bbox, text, conf in results:
            annotations.append({
                'bbox': [[int(x), int(y)] for x, y in bbox],
                'text': text,
                'confidence': float(conf)
            })
        
        # 儲存為 JSON
        output_file = os.path.join(
            output_folder, 
            os.path.splitext(image_file)[0] + '.json'
        )
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump({
                'image': image_file,
                'annotations': annotations
            }, f, ensure_ascii=False, indent=2)
        
        print(f"✓ 已預標註: {image_file} ({len(annotations)} 個文字區域)")

print("""
使用方式:
pre_annotate_images('./dataset/images', './dataset/pre_labels')

然後使用 Label Studio 或其他工具開啟預標註結果,
只需要修正錯誤的部分即可!
""")

---
## 方案 C: 從頭訓練 (完整訓練流程)

⚠️ **難度: 非常高** | 需要大量標註資料 (1000+ 張) | 訓練時間: 數天到數週 | 需要強大 GPU

**除非萬不得已,否則不建議使用此方案!**

方案 A (參數優化 + 預處理 + 後處理) 通常已經可以解決 90% 的問題。

In [ ]:
# 方案 C: 完整訓練流程參考

"""
EasyOCR 完整訓練流程 (僅供參考,非常複雜!)

⚠️ 重要提醒:
1. 需要 1000+ 張標註好的圖片
2. 需要強大的 GPU (至少 RTX 3090 或更好)
3. 訓練時間: 數天到數週
4. 需要深入了解 PyTorch 和深度學習
5. 成本非常高昂

建議: 如果預算允許,直接使用雲端 OCR API 服務!
"""

print("="*60)
print("完整訓練所需資源")
print("="*60)
print("""
硬體需求:
- GPU: RTX 3090 (24GB) 或以上
- RAM: 32GB+
- 儲存空間: 100GB+

軟體需求:
- Python 3.8+
- PyTorch 1.10+
- CUDA 11.3+
- EasyOCR 原始碼

資料需求:
- 訓練集: 1000+ 張標註圖片
- 驗證集: 200+ 張標註圖片
- 測試集: 100+ 張標註圖片

時間成本:
- 資料標註: 40-80 小時
- 模型訓練: 3-7 天
- 調參和優化: 1-2 週

金錢成本:
- GPU 租用: $500-2000
- 人力成本: $2000-5000
- 總計: $2500-7000+
""")

print("\n" + "="*60)
print("訓練步驟 (概述)")
print("="*60)
print("""
1. 準備環境
   git clone https://github.com/JaidedAI/EasyOCR.git
   cd EasyOCR
   pip install -r requirements.txt

2. 準備資料集
   - 格式轉換
   - 資料增強
   - 訓練/驗證/測試集分割

3. 修改配置檔案
   - 語言設定
   - 模型架構
   - 訓練參數

4. 開始訓練
   python train.py --config config.yaml

5. 評估模型
   python eval.py --model_path ./saved_models/best.pth

6. 導出模型
   python export.py --model_path ./saved_models/best.pth
""")

print("\n" + "="*60)
print("替代方案 - 使用雲端 OCR API (推薦!)")
print("="*60)
print("""
如果需要更高的準確率,建議使用專業的 OCR API 服務:

1. **Google Cloud Vision API**
   - 優點: 準確率極高,支援繁體中文
   - 價格: $1.5 / 1000 次 (前 1000 次免費)
   - 網址: https://cloud.google.com/vision

2. **Azure Computer Vision**
   - 優點: 微軟技術,準確率高
   - 價格: $1 / 1000 次 (前 5000 次免費)
   - 網址: https://azure.microsoft.com/services/cognitive-services/computer-vision/

3. **AWS Textract**
   - 優點: 專門針對文件設計
   - 價格: $1.5 / 1000 頁
   - 網址: https://aws.amazon.com/textract/

4. **中華電信 OCR API** (台灣本地)
   - 優點: 針對台灣發票優化
   - 需要洽詢報價
   
性價比分析:
- 自己訓練: $2500-7000 (一次性)
- 使用 API: 每月 1000 張 = $1.5/月

建議: 
如果每月處理量 < 50萬張,使用 API 更划算!
""")

print("\n" + "="*60)
print("總結與建議")
print("="*60)
print("""
推薦的改善流程:

1. 先試 方案 A1-A4 (參數調優 + 預處理 + 後處理 + Ensemble)
   ├─ 成本: 幾乎為零
   ├─ 時間: 數小時
   └─ 效果: 可改善 70-80% 的辨識錯誤

2. 如果還不夠,試 方案 B (Fine-tuning)
   ├─ 成本: $500-1000
   ├─ 時間: 1-2 週
   └─ 效果: 可改善 90-95% 的辨識錯誤

3. 如果仍不滿意,考慮雲端 OCR API
   ├─ 成本: $1-2 / 1000 次
   ├─ 時間: 立即可用
   └─ 效果: 準確率 95-99%

4. 最後才考慮 方案 C (完整訓練)
   ├─ 成本: $2500-7000+
   ├─ 時間: 1-2 個月
   └─ 效果: 取決於資料品質

⭐ 建議: 從方案 A 開始,循序漸進!
""")

---
## 📊 快速開始指南

建議按照以下順序測試各種方法:

### 第一步: 參數優化 (5 分鐘)
執行上面的 **方案 A1** 程式碼,調整 `readtext()` 參數

### 第二步: 預處理優化 (10 分鐘)
執行 **方案 A2** 程式碼,測試不同預處理方法

### 第三步: 後處理修正 (5 分鐘)
執行 **方案 A3** 程式碼,自動修正常見錯誤

### 第四步: 多模型整合 (15 分鐘)
執行 **方案 A4** 程式碼,結合 EasyOCR + DocTR

### 評估效果
比較改善前後的辨識準確率,如果達到需求就完成了!

### 如果還不夠
考慮 **方案 B** (Fine-tuning) 或使用雲端 OCR API

---

💡 **小技巧**: 建議先在 `test.ipynb` 測試各種方法,找到最適合的組合後再整合到正式程式中!

---
## 🔍 針對特定錯字問題的分析

### 您的問題: 
**正確**: 九龍青衣細山路2號美景花園平**臺**72號舖  
**錯誤**: 九龍青衣細山路2號美景花園平**壼**72號舖 或 平**耋臺**72號舖

### 原因分析:
這是 EasyOCR **辨識模型 (Recognition Model)** 的問題,而非文字檢測問題。

EasyOCR 包含兩個部分:
1. **CRAFT (文字檢測)**: 找出文字在哪裡 ✅ (這部分沒問題)
2. **Recognition Model (字符辨識)**: 辨識文字內容 ❌ (這部分出錯)

### EasyOCR 兩個訓練文件說明:

#### 📄 `custom_model.md` - Recognition Model 訓練
- **用途**: 訓練字符辨識模型 (解決您的問題!)
- **解決**: "臺" 被誤認為 "壼" 或 "耋臺" 的問題
- **訓練工具**: 使用 [deep-text-recognition-benchmark](https://github.com/clovaai/deep-text-recognition-benchmark)
- **網路架構**: None-VGG-BiLSTM-CTC (必須是全卷積網路)

#### 📄 `craft/README.md` - CRAFT 文字檢測訓練
- **用途**: 訓練文字位置檢測 (您不需要!)
- **解決**: 找不到文字、文字框不準確的問題
- **您的情況**: 文字檢測正常,不需要訓練 CRAFT

## 💡 針對 "臺" 字的解決方案

### 方案 1: 使用後處理修正 (最簡單!) ⭐

直接在程式中修正這個常見錯誤:

In [ ]:
# 方案 1: 後處理修正 "臺" 字錯誤 (最快速!)

import easyocr

# 擴展錯誤對照表,新增 "臺" 相關錯誤
TAIWAN_CHAR_ERRORS = {
    '壼': '臺',  # 常見錯誤 1
    '耋': '臺',  # 常見錯誤 2 的一部分
    '耋臺': '臺',  # 完整錯誤 2
    '平壼': '平臺',  # 完整詞組修正
    '平耋臺': '平臺',
    '平耋': '平臺',
}

def fix_taiwan_chars(text):
    """修正台灣常見字符錯誤"""
    corrected = text
    
    # 逐一替換錯誤字符
    for wrong, correct in TAIWAN_CHAR_ERRORS.items():
        corrected = corrected.replace(wrong, correct)
    
    return corrected

# 測試修正效果
reader = easyocr.Reader(['ch_tra', 'en'], gpu=True)
image_path = './samples/2.jpeg'

print("="*60)
print("原始 OCR 結果")
print("="*60)
results = reader.readtext(image_path, detail=1)
for bbox, text, conf in results:
    if conf > 0.3:
        print(f"[{conf:.2f}] {text}")

print("\n" + "="*60)
print("修正 '臺' 字錯誤後")
print("="*60)
for bbox, text, conf in results:
    if conf > 0.3:
        corrected = fix_taiwan_chars(text)
        if text != corrected:
            print(f"[{conf:.2f}] {text} → {corrected} ✓ (已修正)")
        else:
            print(f"[{conf:.2f}] {corrected}")

# 完整地址範例測試
print("\n" + "="*60)
print("完整地址修正範例")
print("="*60)
test_addresses = [
    "九龍青衣細山路2號美景花園平壼72號舖",
    "九龍青衣細山路2號美景花園平耋臺72號舖",
    "九龍青衣細山路2號美景花園平耋72號舖",
]

for addr in test_addresses:
    corrected = fix_taiwan_chars(addr)
    print(f"原始: {addr}")
    print(f"修正: {corrected}")
    print(f"正確: {'✓' if '平臺' in corrected else '✗'}")
    print()

---
### 方案 2: 訓練自訂 Recognition Model (根據 custom_model.md)

⚠️ **難度: 高** | 需要標註資料 + GPU | 訓練時間: 數小時到數天

如果後處理無法完全解決,可以訓練自己的辨識模型。

#### 📋 訓練流程 (根據 EasyOCR custom_model.md):

1. **準備資料集**
   - 收集包含 "臺" 字的圖片 (50-200 張)
   - 使用 [TextRecognitionDataGenerator](https://github.com/Belval/TextRecognitionDataGenerator) 生成合成資料
   
2. **訓練模型**
   - 使用 [deep-text-recognition-benchmark](https://github.com/clovaai/deep-text-recognition-benchmark)
   - 或使用 EasyOCR 內建的 [trainer](https://github.com/JaidedAI/EasyOCR/tree/master/trainer)
   - 網路架構: `None-VGG-BiLSTM-CTC`

3. **準備三個檔案**
   - `yourmodel.pth` (訓練好的模型權重)
   - `yourmodel.yaml` (模型配置)
   - `yourmodel.py` (網路架構定義)

4. **使用自訂模型**
   ```python
   reader = easyocr.Reader(['ch_tra', 'en'], recog_network='yourmodel')
   ```

#### 重要檔案位置:
- 網路定義 (`yourmodel.py`) → `~/.EasyOCR/user_network/`
- 模型配置 (`yourmodel.yaml`) → `~/.EasyOCR/user_network/`
- 模型權重 (`yourmodel.pth`) → `~/.EasyOCR/model/`

In [ ]:
# 方案 2: 準備訓練 Recognition Model 的步驟

"""
根據 EasyOCR 的 custom_model.md,這裡是完整的訓練流程
"""

print("="*60)
print("步驟 1: 生成包含 '臺' 字的訓練資料")
print("="*60)
print("""
使用 TextRecognitionDataGenerator 生成合成資料:

安裝:
pip install trdg

生成範例:
trdg -c 1000 -w 5 -f 64 -t 4 \\
     -tc '#000000,#888888' \\
     -b 3 \\
     -d 3 \\
     -k 5 \\
     -rk \\
     -na 2 \\
     -l zh \\
     -i ./custom_dict.txt \\
     -o ./training_data

custom_dict.txt 內容範例:
平臺
臺灣
臺北
臺中
地臺
月臺
櫃臺
... (包含更多 '臺' 字的詞彙)
""")

print("\n" + "="*60)
print("步驟 2: 使用 EasyOCR Trainer 訓練模型")
print("="*60)
print("""
1. Clone EasyOCR repository:
   git clone https://github.com/JaidedAI/EasyOCR.git
   cd EasyOCR/trainer

2. 安裝依賴:
   pip install -r requirements.txt

3. 準備資料集格式:
   training_data/
   ├── gt.txt  (標註檔案)
   └── images/
       ├── img_1.jpg
       ├── img_2.jpg
       └── ...

   gt.txt 格式:
   images/img_1.jpg	平臺
   images/img_2.jpg	臺灣
   images/img_3.jpg	美景花園平臺72號舖

4. 修改配置檔案 (config.yaml):
   train_data: ./training_data
   valid_data: ./validation_data
   select_data: ['/']
   batch_ratio: ['1.0']
   batch_size: 32
   num_iter: 10000
   valInterval: 500
   saved_model: ./saved_models
   character: '你的字符集(包含臺字)'
   
5. 開始訓練:
   python train.py --config config.yaml

6. 訓練完成後會得到:
   saved_models/best_accuracy.pth
""")

print("\n" + "="*60)
print("步驟 3: 準備三個必要檔案")
print("="*60)
print("""
你需要準備三個檔案 (範例在 custom_example.zip):

1. taiwan_custom.pth (訓練好的模型)
2. taiwan_custom.yaml (配置檔案)
   內容:
   ---
   network_params:
     input_channel: 1
     output_channel: 512
     hidden_size: 256
   character: '你的字符集'

3. taiwan_custom.py (網路定義)
   內容:
   from easyocr.model.vgg_model import Model
   # 使用預設的 VGG-BiLSTM-CTC 架構
""")

print("\n" + "="*60)
print("步驟 4: 安裝並使用自訓練模型")
print("="*60)
print("""
1. 將檔案放到指定位置:
   cp taiwan_custom.py ~/.EasyOCR/user_network/
   cp taiwan_custom.yaml ~/.EasyOCR/user_network/
   cp taiwan_custom.pth ~/.EasyOCR/model/

2. 使用自訂模型:
   import easyocr
   reader = easyocr.Reader(
       ['ch_tra', 'en'], 
       recog_network='taiwan_custom',  # 使用你的模型
       gpu=True
   )
   
   results = reader.readtext('./samples/2.jpeg')
   for bbox, text, conf in results:
       print(f"[{conf:.2f}] {text}")
""")

print("\n" + "="*60)
print("⚠️ 重要提醒")
print("="*60)
print("""
1. 訓練 Recognition Model 需要:
   - GPU (至少 RTX 2060 以上)
   - 大量包含 '臺' 字的訓練資料 (建議 500-1000 張)
   - 訓練時間: 數小時到 1-2 天

2. 不需要訓練 CRAFT (文字檢測模型):
   - craft/README.md 是用來訓練文字位置檢測
   - 你的問題是字符辨識,不是文字檢測
   - 所以不需要使用 craft/README.md 的內容

3. 建議優先使用 方案 1 (後處理):
   - 成本: 零
   - 時間: 立即可用
   - 效果: 可解決 90% 的 '臺' 字錯誤
""")

# 實用的字符集生成器
print("\n" + "="*60)
print("字符集準備工具")
print("="*60)

import os

def generate_character_set(data_folder):
    """
    從訓練資料中自動生成字符集
    """
    chars = set()
    
    gt_file = os.path.join(data_folder, 'gt.txt')
    if os.path.exists(gt_file):
        with open(gt_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 2:
                    text = parts[1]
                    chars.update(text)
    
    # 確保包含 '臺' 字
    chars.add('臺')
    
    # 排序並返回
    char_list = sorted(list(chars))
    return ''.join(char_list)

print("""
使用方式:
charset = generate_character_set('./training_data')
print(f"字符集: {charset}")

然後將這個字符集放入 config.yaml 的 'character' 欄位
""")

---
## 📊 兩個 MD 文件的總結對比

| 文件 | 用途 | 是否需要 | 說明 |
|------|------|----------|------|
| **custom_model.md** | 訓練 **Recognition Model**<br>(字符辨識) | ✅ **需要** | 解決 "臺→壼" 的問題<br>訓練字符辨識模型 |
| **craft/README.md** | 訓練 **CRAFT**<br>(文字檢測) | ❌ **不需要** | 用於找文字位置<br>你的檢測已正常 |

### 🎯 針對您的問題建議:

#### 短期解決方案 (立即可用):
1. ✅ **使用方案 1 的後處理修正** (5分鐘完成)
   - 直接替換 `壼 → 臺`、`耋臺 → 臺`
   - 成本: 零
   - 效果: 95%+ 準確率

#### 長期解決方案 (如果要完美):
2. ⚠️ **訓練自訂 Recognition Model** (1-2週)
   - 按照 `custom_model.md` 的步驟
   - 使用 deep-text-recognition-benchmark 或 EasyOCR/trainer
   - 需要 GPU 和訓練資料
   - 成本: $200-500 (GPU租用)
   - 效果: 99%+ 準確率

---

### 💡 實務建議:

**先用方案 1 (後處理)**, 因為:
- 台灣地址中 "臺" 字錯誤是可預測的模式
- 通常出現在: 平臺、臺北、臺灣、臺中等固定詞彙
- 可以建立完整的錯誤對照表
- 無需訓練,立即見效

**只有在以下情況才需要訓練模型**:
- 後處理無法覆蓋所有錯誤情況
- 有大量不同的 "臺" 字使用場景
- 預算和時間充足
- 有 GPU 資源可用

---
# 📚 完整訓練準備流程 (根據 custom_model.md)

## 階段 1️⃣: 環境準備與安裝

In [ ]:
# 階段 1: 檢查環境與安裝必要套件

import sys
import subprocess
import os

print("="*60)
print("步驟 1.1: 檢查 Python 環境")
print("="*60)
print(f"Python 版本: {sys.version}")
print(f"Python 路徑: {sys.executable}")

print("\n" + "="*60)
print("步驟 1.2: 檢查 GPU 可用性")
print("="*60)
try:
    import torch
    print(f"PyTorch 版本: {torch.__version__}")
    print(f"CUDA 可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA 版本: {torch.version.cuda}")
        print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")
        print(f"GPU 數量: {torch.cuda.device_count()}")
    else:
        print("⚠️ 警告: 無 GPU 可用,訓練會非常慢!")
        print("建議: 使用 Google Colab 或雲端 GPU 服務")
except ImportError:
    print("❌ PyTorch 未安裝")
    print("請執行: pip install torch torchvision")

print("\n" + "="*60)
print("步驟 1.3: 安裝訓練所需套件")
print("="*60)

# 必要套件清單
required_packages = {
    'torch': 'PyTorch (深度學習框架)',
    'torchvision': 'PyTorch 視覺套件',
    'Pillow': '圖片處理',
    'lmdb': '資料庫 (訓練資料儲存)',
    'trdg': 'TextRecognitionDataGenerator (生成訓練資料)',
}

print("檢查已安裝的套件:")
for package, description in required_packages.items():
    try:
        __import__(package)
        print(f"✅ {package:20s} - {description}")
    except ImportError:
        print(f"❌ {package:20s} - {description} (未安裝)")

print("\n" + "="*60)
print("安裝指令 (如有套件未安裝,請執行):")
print("="*60)
print("""
# 基礎套件
pip install torch torchvision Pillow lmdb six

# 資料生成工具
pip install trdg

# EasyOCR (如尚未安裝)
pip install easyocr
""")

---
## 階段 2️⃣: Clone 訓練程式碼

根據 custom_model.md,我們需要使用 **deep-text-recognition-benchmark** 或 **EasyOCR/trainer**

In [ ]:
# 階段 2: Clone 訓練程式碼

import os
import subprocess

print("="*60)
print("步驟 2.1: 選擇訓練工具")
print("="*60)
print("""
根據 custom_model.md,有兩個選擇:

選項 A: deep-text-recognition-benchmark (官方推薦)
  - GitHub: https://github.com/clovaai/deep-text-recognition-benchmark
  - 優點: 文檔完整,社群支援好
  - 適合: 初學者

選項 B: EasyOCR/trainer (EasyOCR 官方)
  - GitHub: https://github.com/JaidedAI/EasyOCR/tree/master/trainer
  - 優點: 與 EasyOCR 整合度高
  - 適合: 進階使用者

我們使用 選項 A (deep-text-recognition-benchmark)
""")

print("\n" + "="*60)
print("步驟 2.2: Clone Repository")
print("="*60)

# 設定工作目錄
workspace_dir = os.path.expanduser("~/easyocr_training")
repo_dir = os.path.join(workspace_dir, "deep-text-recognition-benchmark")

print(f"工作目錄: {workspace_dir}")
print(f"Repository 將 clone 到: {repo_dir}")

# 建立指令
clone_commands = f"""
# 建立工作目錄
mkdir -p {workspace_dir}
cd {workspace_dir}

# Clone deep-text-recognition-benchmark
git clone https://github.com/clovaai/deep-text-recognition-benchmark.git
cd deep-text-recognition-benchmark

# 安裝依賴
pip install -r requirements.txt
"""

print("\n請執行以下指令:")
print(clone_commands)

# 檢查是否已經 clone
if os.path.exists(repo_dir):
    print(f"\n✅ Repository 已存在於: {repo_dir}")
else:
    print(f"\n⚠️ Repository 尚未 clone")
    print(f"請在終端機執行上述指令")

---
## 階段 3️⃣: 準備訓練資料

這是最重要的步驟!需要生成包含 "臺" 字的訓練資料。

In [ ]:
# 階段 3: 準備訓練資料 - 建立包含 "臺" 字的詞彙表

import os

print("="*60)
print("步驟 3.1: 建立台灣地址常用詞彙表")
print("="*60)

# 包含 "臺" 字的常用詞彙
taiwan_vocabulary = [
    # 平臺相關
    "平臺", "地臺", "月臺", "櫃臺", "陽臺", "舞臺", "講臺", "臺階",
    
    # 台灣地名
    "臺灣", "臺北", "臺中", "臺南", "臺東", 
    "新北市", "臺北市", "臺中市", "臺南市",
    
    # 常見地址用語
    "一樓平臺", "二樓平臺", "三樓平臺", "地下平臺",
    "美景花園平臺", "陽光大廈平臺", "幸福社區平臺",
    
    # 完整地址範例
    "九龍青衣細山路2號美景花園平臺72號舖",
    "九龍青衣細山路美景花園平臺",
    "美景花園平臺72號",
    "平臺72號舖",
    
    # 其他常見用法
    "臺幣", "臺電", "臺鐵", "臺糖",
    "前臺", "後臺", "柜臺", "臺面",
    
    # 數字組合
    "平臺1號", "平臺2號", "平臺3號", "平臺10號",
    "平臺20號", "平臺30號", "平臺50號", "平臺100號",
]

# 建立資料集目錄
data_dir = os.path.expanduser("~/easyocr_training/training_data")
os.makedirs(data_dir, exist_ok=True)

# 儲存詞彙表
vocab_file = os.path.join(data_dir, "taiwan_vocab.txt")
with open(vocab_file, 'w', encoding='utf-8') as f:
    for word in taiwan_vocabulary:
        f.write(word + '\n')

print(f"✅ 已建立詞彙表: {vocab_file}")
print(f"📊 總共 {len(taiwan_vocabulary)} 個詞彙")
print(f"\n前 20 個詞彙:")
for i, word in enumerate(taiwan_vocabulary[:20], 1):
    print(f"  {i:2d}. {word}")

print("\n" + "="*60)
print("步驟 3.2: 準備字符集 (Character Set)")
print("="*60)

# 收集所有字符
all_chars = set()
for word in taiwan_vocabulary:
    all_chars.update(word)

# 加入數字和標點符號
all_chars.update('0123456789')
all_chars.update('號舖樓室座棟巷弄路街段區市縣鄉鎮村里鄰')

# 排序
char_list = sorted(list(all_chars))
charset = ''.join(char_list)

print(f"字符集大小: {len(char_list)} 個字符")
print(f"字符集: {charset}")

# 儲存字符集
charset_file = os.path.join(data_dir, "character_set.txt")
with open(charset_file, 'w', encoding='utf-8') as f:
    f.write(charset)

print(f"\n✅ 已儲存字符集: {charset_file}")

# 檢查 '臺' 字是否在字符集中
if '臺' in char_list:
    print("✅ '臺' 字已包含在字符集中")
else:
    print("❌ 錯誤: '臺' 字不在字符集中!")

In [ ]:
# 階段 3.3: 使用 TextRecognitionDataGenerator 生成訓練圖片

print("="*60)
print("步驟 3.3: 生成訓練圖片")
print("="*60)

import os
import subprocess

# 設定路徑
data_dir = os.path.expanduser("~/easyocr_training/training_data")
vocab_file = os.path.join(data_dir, "taiwan_vocab.txt")
output_dir = os.path.join(data_dir, "generated_images")

os.makedirs(output_dir, exist_ok=True)

print(f"詞彙表: {vocab_file}")
print(f"輸出目錄: {output_dir}")

# 生成指令
trdg_command = f"""
# 使用 trdg 生成訓練圖片
trdg -c 2000 \\
     -w 1 \\
     -f 64 \\
     -t 4 \\
     -k 5 \\
     -rk \\
     -bl 2 \\
     -rbl \\
     -na 2 \\
     -tc '#000000,#333333,#666666' \\
     -d 3 \\
     -do \\
     -k 5 \\
     -i {vocab_file} \\
     -o {output_dir}

參數說明:
-c 2000      : 生成 2000 張圖片
-w 1         : 每張圖片 1 個詞
-f 64        : 字體大小 64
-t 4         : 使用 4 個執行緒
-k 5         : 傾斜角度 ±5 度
-rk          : 隨機傾斜
-bl 2        : 模糊程度 2
-rbl         : 隨機模糊
-na 2        : 添加噪點
-tc '#...'   : 文字顏色 (黑、深灰、灰)
-d 3         : 扭曲程度
-do          : 啟用扭曲
-i           : 輸入詞彙表
-o           : 輸出目錄
"""

print("\n請在終端機執行以下指令:")
print(trdg_command)

print("\n" + "="*60)
print("或使用 Python 直接執行:")
print("="*60)

# 使用 Python API 生成 (需要安裝 trdg)
try:
    from trdg.generators import GeneratorFromStrings
    
    print("正在使用 trdg 生成圖片...")
    
    # 讀取詞彙
    with open(vocab_file, 'r', encoding='utf-8') as f:
        strings = [line.strip() for line in f if line.strip()]
    
    # 生成器設定
    generator = GeneratorFromStrings(
        strings=strings,
        count=min(len(strings) * 3, 2000),  # 每個詞生成 3 張,最多 2000 張
        fonts=['STHeiti', 'Arial Unicode MS'],  # Mac 中文字體
        size=64,
        skewing_angle=5,
        random_skew=True,
        blur=2,
        random_blur=True,
        background_type=0,  # 純色背景
        distorsion_type=2,  # 扭曲類型
    )
    
    # 生成圖片
    count = 0
    for img, lbl in generator:
        img_path = os.path.join(output_dir, f"img_{count:05d}.jpg")
        lbl_path = os.path.join(output_dir, f"img_{count:05d}.txt")
        
        # 儲存圖片
        img.save(img_path)
        
        # 儲存標籤
        with open(lbl_path, 'w', encoding='utf-8') as f:
            f.write(lbl)
        
        count += 1
        if count % 100 == 0:
            print(f"已生成 {count} 張圖片...")
        
        if count >= 2000:
            break
    
    print(f"\n✅ 成功生成 {count} 張訓練圖片!")
    print(f"📁 圖片位置: {output_dir}")
    
except ImportError:
    print("❌ trdg 未安裝")
    print("請執行: pip install trdg")
    print("然後在終端機執行上述指令")
except Exception as e:
    print(f"⚠️ 生成失敗: {e}")
    print("請改用終端機指令生成")

In [ ]:
# 階段 3.4: 轉換資料格式為 LMDB (deep-text-recognition-benchmark 格式)

print("="*60)
print("步驟 3.4: 轉換資料為 LMDB 格式")
print("="*60)

import os
import lmdb
import cv2
import numpy as np
from PIL import Image
import six

def create_lmdb_dataset(input_dir, output_dir, charset_file):
    """
    將生成的圖片轉換為 LMDB 格式
    
    Args:
        input_dir: 包含圖片和標籤的目錄
        output_dir: LMDB 資料庫輸出目錄
        charset_file: 字符集檔案
    """
    
    os.makedirs(output_dir, exist_ok=True)
    
    # 讀取字符集
    with open(charset_file, 'r', encoding='utf-8') as f:
        charset = f.read().strip()
    
    print(f"輸入目錄: {input_dir}")
    print(f"輸出目錄: {output_dir}")
    print(f"字符集: {len(charset)} 個字符")
    
    # 收集所有圖片
    image_files = []
    for fname in sorted(os.listdir(input_dir)):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            txt_file = fname.rsplit('.', 1)[0] + '.txt'
            txt_path = os.path.join(input_dir, txt_file)
            if os.path.exists(txt_path):
                image_files.append(fname)
    
    print(f"找到 {len(image_files)} 張有標籤的圖片")
    
    if len(image_files) == 0:
        print("❌ 錯誤: 沒有找到任何圖片!")
        return
    
    # 建立 LMDB 資料庫
    env = lmdb.open(output_dir, map_size=1099511627776)  # 1TB
    
    with env.begin(write=True) as txn:
        cnt = 0
        for img_fname in image_files:
            # 讀取圖片
            img_path = os.path.join(input_dir, img_fname)
            img = Image.open(img_path)
            
            # 轉為 bytes
            img_byte_arr = np.array(img)
            _, img_encoded = cv2.imencode('.jpg', img_byte_arr)
            img_bytes = img_encoded.tobytes()
            
            # 讀取標籤
            txt_fname = img_fname.rsplit('.', 1)[0] + '.txt'
            txt_path = os.path.join(input_dir, txt_fname)
            with open(txt_path, 'r', encoding='utf-8') as f:
                label = f.read().strip()
            
            # 寫入 LMDB
            imageKey = f'image-{cnt+1:09d}'.encode()
            labelKey = f'label-{cnt+1:09d}'.encode()
            
            txn.put(imageKey, img_bytes)
            txn.put(labelKey, label.encode('utf-8'))
            
            cnt += 1
            if cnt % 100 == 0:
                print(f"已處理 {cnt}/{len(image_files)} 張圖片...")
        
        # 寫入統計資訊
        txn.put('num-samples'.encode(), str(cnt).encode())
    
    env.close()
    print(f"\n✅ LMDB 資料庫建立完成!")
    print(f"   總共 {cnt} 張圖片")
    print(f"   位置: {output_dir}")
    
    return cnt

# 執行轉換
data_dir = os.path.expanduser("~/easyocr_training/training_data")
input_dir = os.path.join(data_dir, "generated_images")
train_lmdb = os.path.join(data_dir, "train_lmdb")
val_lmdb = os.path.join(data_dir, "val_lmdb")
charset_file = os.path.join(data_dir, "character_set.txt")

if os.path.exists(input_dir) and os.listdir(input_dir):
    try:
        # 生成訓練集 (80%) 和驗證集 (20%)
        all_files = [f for f in os.listdir(input_dir) if f.endswith('.jpg')]
        n_total = len(all_files)
        n_train = int(n_total * 0.8)
        
        print(f"\n總圖片數: {n_total}")
        print(f"訓練集: {n_train} 張 (80%)")
        print(f"驗證集: {n_total - n_train} 張 (20%)")
        
        # 分割檔案
        import random
        random.shuffle(all_files)
        train_files = all_files[:n_train]
        val_files = all_files[n_train:]
        
        # 建立臨時目錄
        train_tmp = os.path.join(data_dir, "train_tmp")
        val_tmp = os.path.join(data_dir, "val_tmp")
        os.makedirs(train_tmp, exist_ok=True)
        os.makedirs(val_tmp, exist_ok=True)
        
        # 複製檔案到臨時目錄
        import shutil
        print("\n複製訓練集檔案...")
        for f in train_files:
            shutil.copy(os.path.join(input_dir, f), train_tmp)
            txt_f = f.rsplit('.', 1)[0] + '.txt'
            if os.path.exists(os.path.join(input_dir, txt_f)):
                shutil.copy(os.path.join(input_dir, txt_f), train_tmp)
        
        print("複製驗證集檔案...")
        for f in val_files:
            shutil.copy(os.path.join(input_dir, f), val_tmp)
            txt_f = f.rsplit('.', 1)[0] + '.txt'
            if os.path.exists(os.path.join(input_dir, txt_f)):
                shutil.copy(os.path.join(input_dir, txt_f), val_tmp)
        
        # 建立 LMDB
        print("\n" + "="*60)
        print("建立訓練集 LMDB")
        print("="*60)
        create_lmdb_dataset(train_tmp, train_lmdb, charset_file)
        
        print("\n" + "="*60)
        print("建立驗證集 LMDB")
        print("="*60)
        create_lmdb_dataset(val_tmp, val_lmdb, charset_file)
        
        # 清理臨時目錄
        shutil.rmtree(train_tmp)
        shutil.rmtree(val_tmp)
        
        print("\n✅ 資料準備完成!")
        
    except Exception as e:
        print(f"❌ 錯誤: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"⚠️ 請先執行上一個 cell 生成圖片")
    print(f"   或確認目錄存在: {input_dir}")

---
## 階段 4️⃣: 配置訓練參數

根據 custom_model.md,網路架構必須是 **None-VGG-BiLSTM-CTC**

In [ ]:
# 階段 4: 建立訓練配置

import os

print("="*60)
print("步驟 4.1: 生成訓練指令")
print("="*60)

# 路徑設定
data_dir = os.path.expanduser("~/easyocr_training/training_data")
train_lmdb = os.path.join(data_dir, "train_lmdb")
val_lmdb = os.path.join(data_dir, "val_lmdb")
charset_file = os.path.join(data_dir, "character_set.txt")
repo_dir = os.path.expanduser("~/easyocr_training/deep-text-recognition-benchmark")
saved_models = os.path.join(data_dir, "saved_models")

# 讀取字符集
try:
    with open(charset_file, 'r', encoding='utf-8') as f:
        charset = f.read().strip()
    print(f"✅ 字符集載入成功: {len(charset)} 個字符")
except:
    print(f"❌ 無法載入字符集: {charset_file}")
    charset = ""

# 根據 custom_model.md,必須使用 None-VGG-BiLSTM-CTC
training_command = f"""
cd {repo_dir}

# 訓練指令 (根據 custom_model.md)
python train.py \\
    --train_data {train_lmdb} \\
    --valid_data {val_lmdb} \\
    --select_data / \\
    --batch_ratio 1.0 \\
    --Transformation None \\
    --FeatureExtraction VGG \\
    --SequenceModeling BiLSTM \\
    --Prediction CTC \\
    --batch_size 32 \\
    --num_iter 10000 \\
    --valInterval 500 \\
    --saved_model {saved_models} \\
    --character '{charset}' \\
    --manualSeed 1111 \\
    --workers 4 \\
    --lr 1.0 \\
    --adam

參數說明:
--Transformation None    : 不使用轉換 (必須!)
--FeatureExtraction VGG  : 使用 VGG 特徵提取 (必須!)
--SequenceModeling BiLSTM: 使用 BiLSTM 序列建模 (必須!)
--Prediction CTC         : 使用 CTC 預測 (必須!)
--batch_size 32          : 批次大小 (根據 GPU 記憶體調整)
--num_iter 10000         : 訓練迭代次數
--valInterval 500        : 每 500 次驗證一次
--character              : 字符集 (包含 '臺' 字)
--lr 1.0                 : 學習率
--adam                   : 使用 Adam 優化器
"""

print("\n請在終端機執行以下訓練指令:")
print(training_command)

print("\n" + "="*60)
print("步驟 4.2: GPU 記憶體不足時的調整")
print("="*60)
print("""
如果訓練時出現 CUDA out of memory 錯誤:

1. 降低 batch_size:
   --batch_size 16  (或 8, 4)

2. 降低圖片大小:
   --imgH 32 --imgW 100

3. 使用梯度累積:
   --grad_clip 5

4. 使用 mixed precision training:
   --fp16
""")

print("\n" + "="*60)
print("步驟 4.3: 預期訓練時間")
print("="*60)
print("""
訓練時間預估 (10000 iterations):
- RTX 4090:  2-3 小時
- RTX 3090:  4-6 小時
- RTX 2060:  8-12 小時
- 無 GPU:    1-2 天 (不建議)

Google Colab (免費 GPU):
- T4 GPU:    6-8 小時
- 建議使用 Colab Pro (V100/A100 GPU)
""")

---
## 階段 5️⃣: 訓練後的模型準備

訓練完成後,需要準備 3 個檔案來使用自訂模型 (根據 custom_model.md)

In [ ]:
# 階段 5: 準備 EasyOCR 自訂模型所需的三個檔案

import os

print("="*60)
print("訓練完成後的步驟")
print("="*60)
print("""
訓練完成後,會在 saved_models/ 目錄下得到:
- best_accuracy.pth (最佳模型權重)
- iter_10000.pth (最終迭代模型)

接下來需要準備 3 個檔案給 EasyOCR 使用
""")

# 檔案路徑
easyocr_user_network = os.path.expanduser("~/.EasyOCR/user_network")
easyocr_model_dir = os.path.expanduser("~/.EasyOCR/model")
data_dir = os.path.expanduser("~/easyocr_training/training_data")
charset_file = os.path.join(data_dir, "character_set.txt")

os.makedirs(easyocr_user_network, exist_ok=True)
os.makedirs(easyocr_model_dir, exist_ok=True)

print("\n" + "="*60)
print("檔案 1: taiwan_custom.py (網路定義)")
print("="*60)

# 建立網路定義檔案
network_code = '''"""
Taiwan Custom Recognition Model
Architecture: None-VGG-BiLSTM-CTC (根據 custom_model.md)
"""

import torch.nn as nn

class Model(nn.Module):
    def __init__(self, input_channel, output_channel, hidden_size, num_class):
        super(Model, self).__init__()
        
        # VGG Feature Extractor
        self.FeatureExtraction = VGG_FeatureExtractor(input_channel, output_channel)
        self.FeatureExtraction_output = output_channel
        self.AdaptiveAvgPool = nn.AdaptiveAvgPool2d((None, 1))
        
        # BiLSTM Sequence Modeling
        self.SequenceModeling = nn.Sequential(
            BidirectionalLSTM(output_channel, hidden_size, hidden_size),
            BidirectionalLSTM(hidden_size, hidden_size, hidden_size)
        )
        self.SequenceModeling_output = hidden_size
        
        # CTC Prediction
        self.Prediction = nn.Linear(self.SequenceModeling_output, num_class)

    def forward(self, input, text):
        # Feature extraction
        visual_feature = self.FeatureExtraction(input)
        visual_feature = self.AdaptiveAvgPool(visual_feature.permute(0, 3, 1, 2))
        visual_feature = visual_feature.squeeze(3)
        
        # Sequence modeling
        contextual_feature = self.SequenceModeling(visual_feature)
        
        # Prediction
        prediction = self.Prediction(contextual_feature.contiguous())
        
        return prediction


class VGG_FeatureExtractor(nn.Module):
    def __init__(self, input_channel, output_channel=512):
        super(VGG_FeatureExtractor, self).__init__()
        self.output_channel = [int(output_channel / 8), int(output_channel / 4),
                               int(output_channel / 2), output_channel]
        
        self.ConvNet = nn.Sequential(
            nn.Conv2d(input_channel, self.output_channel[0], 3, 1, 1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(self.output_channel[0], self.output_channel[1], 3, 1, 1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(self.output_channel[1], self.output_channel[2], 3, 1, 1), nn.ReLU(True),
            nn.Conv2d(self.output_channel[2], self.output_channel[2], 3, 1, 1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(self.output_channel[2], self.output_channel[3], 3, 1, 1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),
            nn.Conv2d(self.output_channel[3], self.output_channel[3], 3, 1, 1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(self.output_channel[3], self.output_channel[3], 2, 1, 0), nn.ReLU(True)
        )

    def forward(self, input):
        return self.ConvNet(input)


class BidirectionalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BidirectionalLSTM, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(hidden_size * 2, output_size)

    def forward(self, input):
        self.rnn.flatten_parameters()
        recurrent, _ = self.rnn(input)
        output = self.linear(recurrent)
        return output
'''

network_file = os.path.join(easyocr_user_network, "taiwan_custom.py")
with open(network_file, 'w', encoding='utf-8') as f:
    f.write(network_code)

print(f"✅ 已建立: {network_file}")

print("\n" + "="*60)
print("檔案 2: taiwan_custom.yaml (模型配置)")
print("="*60)

# 讀取字符集
with open(charset_file, 'r', encoding='utf-8') as f:
    charset = f.read().strip()

yaml_content = f'''network_params:
  input_channel: 1
  output_channel: 512
  hidden_size: 256

character: '{charset}'

model_name: 'taiwan_custom'
'''

yaml_file = os.path.join(easyocr_user_network, "taiwan_custom.yaml")
with open(yaml_file, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"✅ 已建立: {yaml_file}")
print(f"   字符集包含 {len(charset)} 個字符")

print("\n" + "="*60)
print("檔案 3: taiwan_custom.pth (模型權重)")
print("="*60)
print(f"""
訓練完成後,將 best_accuracy.pth 複製到:
{easyocr_model_dir}/taiwan_custom.pth

指令:
cp ~/easyocr_training/training_data/saved_models/best_accuracy.pth \\
   {easyocr_model_dir}/taiwan_custom.pth
""")

print("\n" + "="*60)
print("檔案位置總結")
print("="*60)
print(f"""
1. taiwan_custom.py   → {network_file}
2. taiwan_custom.yaml → {yaml_file}
3. taiwan_custom.pth  → {easyocr_model_dir}/taiwan_custom.pth (訓練後複製)

三個檔案必須同名 (taiwan_custom),但副檔名不同!
""")

---
## 階段 6️⃣: 使用自訓練模型

訓練完成並準備好 3 個檔案後,就可以使用自訂模型了!

In [ ]:
# 階段 6: 使用自訓練的模型進行 OCR

import easyocr
import os

print("="*60)
print("使用自訓練模型")
print("="*60)

# 檢查檔案是否存在
easyocr_user_network = os.path.expanduser("~/.EasyOCR/user_network")
easyocr_model_dir = os.path.expanduser("~/.EasyOCR/model")

required_files = {
    'taiwan_custom.py': os.path.join(easyocr_user_network, 'taiwan_custom.py'),
    'taiwan_custom.yaml': os.path.join(easyocr_user_network, 'taiwan_custom.yaml'),
    'taiwan_custom.pth': os.path.join(easyocr_model_dir, 'taiwan_custom.pth'),
}

print("檢查必要檔案:")
all_ready = True
for name, path in required_files.items():
    exists = os.path.exists(path)
    status = "✅" if exists else "❌"
    print(f"{status} {name:25s} - {path}")
    if not exists:
        all_ready = False

if all_ready:
    print("\n✅ 所有檔案就緒,可以使用自訓練模型!")
    
    try:
        print("\n" + "="*60)
        print("載入自訓練模型")
        print("="*60)
        
        # 使用自訓練模型
        reader = easyocr.Reader(
            ['ch_tra', 'en'],
            recog_network='taiwan_custom',  # 使用自訓練模型
            gpu=True
        )
        
        print("✅ 模型載入成功!")
        
        # 測試辨識
        print("\n" + "="*60)
        print("測試自訓練模型")
        print("="*60)
        
        image_path = './samples/2.jpeg'
        
        if os.path.exists(image_path):
            results = reader.readtext(image_path, detail=1)
            
            print("辨識結果:")
            for bbox, text, conf in results:
                if conf > 0.3:
                    # 檢查是否正確辨識 '臺' 字
                    has_tai = '臺' in text
                    marker = "✅" if has_tai else ""
                    print(f"[{conf:.2f}] {text} {marker}")
            
            # 檢查完整地址
            full_text = ' '.join([text for _, text, conf in results if conf > 0.3])
            if '平臺' in full_text or '臺' in full_text:
                print("\n✅ 成功辨識 '臺' 字!")
            else:
                print("\n⚠️ 未辨識到 '臺' 字")
        else:
            print(f"⚠️ 測試圖片不存在: {image_path}")
            
    except Exception as e:
        print(f"❌ 錯誤: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("\n⚠️ 請先完成訓練並準備好所有檔案")
    print("\n缺少的檔案:")
    for name, path in required_files.items():
        if not os.path.exists(path):
            print(f"  - {path}")

print("\n" + "="*60)
print("對比測試")
print("="*60)
print("""
如果要對比原始模型和自訓練模型:

# 原始模型
reader_original = easyocr.Reader(['ch_tra', 'en'], gpu=True)
results_original = reader_original.readtext('./samples/2.jpeg')

# 自訓練模型
reader_custom = easyocr.Reader(['ch_tra', 'en'], recog_network='taiwan_custom', gpu=True)
results_custom = reader_custom.readtext('./samples/2.jpeg')

# 比較結果
print("原始模型:", results_original)
print("自訓練模型:", results_custom)
""")

---
## 📋 完整流程檢查清單

### ✅ 準備階段
- [ ] 安裝 PyTorch 和 CUDA (如有 GPU)
- [ ] 安裝 trdg (TextRecognitionDataGenerator)
- [ ] 安裝 lmdb, Pillow, easyocr
- [ ] Clone deep-text-recognition-benchmark

### ✅ 資料準備階段
- [ ] 建立台灣詞彙表 (包含 '臺' 字)
- [ ] 生成字符集 (確認包含 '臺')
- [ ] 使用 trdg 生成 2000+ 張訓練圖片
- [ ] 轉換為 LMDB 格式
- [ ] 分割訓練集 (80%) 和驗證集 (20%)

### ✅ 訓練階段
- [ ] 配置訓練參數 (None-VGG-BiLSTM-CTC)
- [ ] 執行訓練指令
- [ ] 監控訓練進度 (驗證準確率)
- [ ] 等待訓練完成 (10000 iterations)

### ✅ 部署階段
- [ ] 建立 taiwan_custom.py (網路定義)
- [ ] 建立 taiwan_custom.yaml (模型配置)
- [ ] 複製 best_accuracy.pth → taiwan_custom.pth
- [ ] 測試自訓練模型
- [ ] 驗證 '臺' 字辨識準確率

---

## ⏱️ 預估時間軸

| 階段 | 時間 | 說明 |
|------|------|------|
| 環境準備 | 30分鐘 | 安裝套件、clone repository |
| 資料準備 | 1-2小時 | 生成圖片、轉換格式 |
| 訓練 | 4-12小時 | 取決於 GPU (或使用 Colab) |
| 部署測試 | 30分鐘 | 準備檔案、測試模型 |
| **總計** | **6-15小時** | 不含等待訓練時間 |

---

## 💰 成本估算

### 選項 A: 使用自己的 GPU
- 電費: 約 $2-5 (訓練 8-12 小時)
- 總成本: **$2-5**

### 選項 B: 使用雲端 GPU
- Google Colab Pro: $10/月 (V100 GPU, 24小時連續)
- AWS p3.2xlarge: $3.06/小時 × 8小時 = **$24.48**
- Azure NC6: $0.90/小時 × 8小時 = **$7.20**

### 選項 C: 直接使用後處理 (推薦!)
- 成本: **$0**
- 時間: **5分鐘**
- 效果: **95%+ 準確率**

---

## 🎯 最終建議

1. **立即可用**: 先執行本 notebook 開頭的「方案 1: 後處理修正」
   - 零成本、立即見效
   - 可解決大部分 '臺' 字問題

2. **短期方案**: 如果後處理不夠完美
   - 增加更多錯誤對照規則
   - 使用多模型整合 (Ensemble)

3. **長期方案**: 只在以下情況才訓練模型
   - 後處理無法覆蓋所有情況
   - 需要處理大量不同的地址格式
   - 有充足的時間和預算
   - 有 GPU 資源或願意使用雲端服務

💡 **實務上,90% 的情況使用後處理就足夠了!**